<a href="https://colab.research.google.com/github/NattachaiWat/ml442/blob/batch4/Lab1_1_face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dlib

In [ ]:
# install dlib
!apt update
!apt install -y cmake
!pip install dlib

In [ ]:
!pip install face_recognition

In [ ]:
import cv2
import face_recognition
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
from os import listdir
from os.path import join

# Download example

In [ ]:
%%bash
wget https://storage.googleapis.com/nattachai-ai-data/share/lab1-1_images.zip
unzip lab1-1_images.zip

# Face Detection


In [ ]:
# Load the jpg file into a numpy array
image = face_recognition.load_image_file("/content/lab1.1_images/faces.jpg")
disp_img = image.copy()

# Find all the faces in the image using a pre-trained convolutional neural network.
face_locations = face_recognition.face_locations(image, 
                                                 number_of_times_to_upsample=0, 
                                                 model="cnn")
print(face_locations)

In [ ]:
# Display face detection result
for face_location in face_locations:
    top, right, bottom, left = face_location
    cv2.rectangle(disp_img,(left,top),(right,bottom),(0,255,0),3)
    
print("There are {} face(s) in this picture.".format(len(face_locations)))
plt.imshow(disp_img)
plt.show()

# Facial Landmarks

In [ ]:
# Load image and find face landmarks
image = face_recognition.load_image_file("/content/lab1.1_images/faces.jpg")
face_landmarks_list = face_recognition.face_landmarks(image)

print(face_landmarks_list)

In [ ]:
# Display results

pil_image = Image.fromarray(image)
# Creates an object that can be used to draw in the given image.
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:

    # Print the location of each facial feature in this image
    for facial_feature in face_landmarks.keys():
        print("{}: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in face_landmarks.keys():
        d.line(face_landmarks[facial_feature], width=5)

plt.imshow(pil_image)
plt.show()

# Face Verification

In [ ]:
# Crop face
face_img = face_recognition.load_image_file("/content/lab1.1_images/known_people/Nadet.jpg")
face_locations = face_recognition.face_locations(face_img,
                                           number_of_times_to_upsample=0,
                                           model="cnn")
top, right, bottom, left = face_locations[0]
cropped = face_img[top:bottom, left:right]

plt.imshow(cropped)
plt.show()

In [ ]:
# Find face encoding
face_encoding = face_recognition.face_encodings(cropped)[0]

print('Size of encoding: ',len(face_encoding))

In [ ]:
# Find all faces in an image
image = face_recognition.load_image_file("/content/lab1.1_images/faces.jpg")
plt.imshow(image)
plt.show()

face_locations = face_recognition.face_locations(image, 
                                                 number_of_times_to_upsample=0, 
                                                 model="cnn")
print(face_locations)

In [ ]:
# Find face encoding and compare
for face_location in face_locations:
    top, right, bottom, left = face_location
    unknown_face = image[top:bottom, left:right]
    unknown_face_encoding = face_recognition.face_encodings(unknown_face)[0]

    # Compare two face encodings
    results = face_recognition.compare_faces([face_encoding], unknown_face_encoding)

    plt.imshow(unknown_face)
    plt.show()

    if results[0] == True:
        print("This is Nadet")
    else:
        print("This is not Nadet")

# Face Recognition

In [ ]:
# Find face encoding for all known people
filename = [f for f in listdir('/content/lab1.1_images/known_people')]

known_faces = []
known_faces_name = []

for fname in filename:
    image = face_recognition.load_image_file(os.path.join('/content/lab1.1_images/known_people',fname))
    face_encoding = face_recognition.face_encodings(image)[0]
    known_faces.append(face_encoding)
    known_faces_name.append(fname[:-4])

print(len(known_faces),' known faces: ',known_faces_name)

In [ ]:
# Find all face locations in a test image
image = face_recognition.load_image_file("/content/lab1.1_images/faces.jpg")
disp_img = image.copy()
face_locations = face_recognition.face_locations(image, 
                                                 number_of_times_to_upsample=0, 
                                                 model="cnn")

In [ ]:
# Find face encoding and compare with known face encoding
for face_location in face_locations:
    top, right, bottom, left = face_location
    cv2.rectangle(disp_img,(left,top),(right,bottom),(0,255,0),3)
    
    # crop face image
    unknown_face = image[top:bottom, left:right]

    # find face encoding
    unknown_face_encoding = face_recognition.face_encodings(unknown_face)[0]

    # find a match
    match = face_recognition.compare_faces(known_faces, unknown_face_encoding)
    cv2.putText(disp_img, known_faces_name[match.index(True)], (left,top), 
                cv2.FONT_HERSHEY_SIMPLEX,  
                fontScale = 2, 
                color = (255, 0, 0), 
                thickness = 3) 

plt.imshow(disp_img)
plt.show()

In [ ]:
# Show distance between face encoding
face_distances = face_recognition.face_distance(known_faces, 
                                                unknown_face_encoding)
print(known_faces_name)
print(match)
print(face_distances)

# The Last One

In [ ]:
!face_recognition /content/lab1.1_images/known_people/ /content/lab1.1_images/test1.jpg

In [ ]:
!face_recognition --show-distance true /content/lab1.1_images/known_people/ /content/lab1.1_images/test1.jpg

In [ ]:
!face_recognition --show-distance true --tolerance 0.5 /content/lab1.1_images/known_people/ /content/lab1.1_images/test1.jpg